In [ ]:
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

In [ ]:
def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.

    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=4):
    """
    NOTE: this is the function you might want to use as a starting point once you want to
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).

    Think about things like separating line segments by their
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of
    the lines and extrapolate to the top and bottom of the lane.

    This function draws `lines` with `color` and `thickness`.
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.

    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), min_line_len, max_line_gap)
    #line_img = np.zeros((img.shape), dype = numpy.uint)
    #line_img = np.copy(img) * 0
    #line_img = draw_lines(line_img, lines)
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1, λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.

    `initial_img` should be the image before any processing.

    The result image is computed as follows:

    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    gs_img = grayscale(image)
    gblur_img= gaussian_blur(gs_img, image_params['kz'])
    canny_img = canny(gblur_img, image_params['lt'], image_params['ht'])
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(0, 0), (imshape[1], 0), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_img = region_of_interest(canny_img, vertices)
    lines = hough_lines(masked_img, image_params['rho'], image_params['theta'], image_params['thrsh'], image_params['minlt'],image_params['maxlg'])
    line_img = np.copy(image) * 0
    draw_lines(line_img, lines)
    image = weighted_img(line_img, image)
    return image

In [ ]:
param = dict(
[
('solidWhiteRight.jpg', {'kz':11,'lt':190, 'ht':230, 'rho':0.7, 'theta':np.pi/320, 'thrsh':30,'minlt': 20, 'maxlg':30}),
('solidWhiteCurve.jpg',  {'kz':9,'lt':180, 'ht':190, 'rho':0.7, 'theta':np.pi/320, 'thrsh':40,'minlt':20, 'maxlg':30}),
('solidYellowCurve.jpg', {'kz':5,'lt':200, 'ht':250, 'rho':1, 'theta':np.pi/180, 'thrsh':5,'minlt':120, 'maxlg':50}),
('solidYellowCurve2.jpg', {'kz':5,'lt':200, 'ht':250, 'rho':1, 'theta':np.pi/180, 'thrsh':100,'minlt':100, 'maxlg':90}),
('solidYellowLeft.jpg', {'kz':9,'lt':140, 'ht':200, 'rho':0.8, 'theta':np.pi/300, 'thrsh':50,'minlt':25, 'maxlg':60}),
])

for key, value in param.items():
    image = (mpimg.imread("test_images/"+key))
    image_params = value
    image = process_image(image)
    plt.title(key)
    plt.imsave("test_images/out_"+key, image)


In [ ]:
params_vid = dict([
('solidWhiteRight.mp4', {'kz':9,'lt':220, 'ht':230, 'rho':0.7, 'theta':np.pi/220, 'thrsh':30,'minlt': 25, 'maxlg':35}),
('solidYellowLeft.mp4', {'kz':11,'lt':130, 'ht':150, 'rho':1, 'theta':np.pi/180, 'thrsh':120,'minlt':20, 'maxlg':35}),
])

for key, value in params_vid.items():
    image_params = value
    output = "test_images/out_"+key
    clip = VideoFileClip("test_images/"+key)
    processed_clip = clip.fl_image(process_image)
    processed_clip.write_videofile(output, audio=False)